In [1]:
!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmppdo1o9lu".


In [11]:
%%cuda
// версия 2
// как в методичке


#include <iostream>
#include <vector>
#include <algorithm>
#include <random>
#include <cmath>
#include <limits>
#include <chrono>


const double left = -5.0; // граница слева
const double right = 5.0; // граница справа
const int number_points = 500; // количество точек
const double step = (right - left)/number_points; // шаг для x
const int POPULATION_SIZE = 2000; // Размер популяции
const int MAX_GENERATIONS = 1000; // Максимальное количество поколений
const double MUTATION_RATE = 0.01; // Вероятность мутации
const double TARGET_ERROR = 1e-5; // целевая ошибка
const int POLYNOMIAL_DEGREE = 4; // степень полинома
std::vector<double> TARGET_POLYNOM(POLYNOMIAL_DEGREE+1);
const double COEF_SELECTION = 0.5;
const int maxConstIter = 50;


std::vector<double> POINTS;
std::vector<double> FUNCTION_VALUES;

std::random_device rd;
std::mt19937 gen(rd());
std::uniform_real_distribution<> dis(left, right);
std::uniform_real_distribution<> distr(-1.0, 1.0);

// Тип для хранения коэффициентов полинома
using Individual = std::vector<double>;

// Подсчёт значения полинома в точке x для данных коэффициентов
double function_value(const Individual& individual, double x) {
    double result = 0.0;
    for (int i = 0; i <= POLYNOMIAL_DEGREE; ++i) { result += individual[i] * std::pow(x, i); }
    return result;
}

void initFuncValues(){
    for(double& value: TARGET_POLYNOM) value = dis(gen);
    for(double point = left; point < right; point+= step){
        POINTS.push_back(point);
        FUNCTION_VALUES.push_back(function_value(TARGET_POLYNOM,point));
    }
}

// Функция оценки пригодности (fitness) для каждого индивидуума, в целом не используется, так как использую сортировку, и просто там используется функция подсчета ошибки
std::vector<double> Fitness(const std::vector<Individual>& population) {
    std::vector<double> fitnesses(population.size());
    for (int i = 0; i < population.size(); ++i) {
        double sumError = 0.0;
        for (int j = 0; j < POINTS.size(); ++j) {
            double f_approx = function_value(population[i], POINTS[j]);
            sumError += std::pow(FUNCTION_VALUES[j] - f_approx, 2);
        }
        fitnesses[i] = sumError / POINTS.size();
    }
    return fitnesses;
}

// Скрещивание (crossover) двух родителей для создания двух детей
std::vector<Individual> Crossover(std::vector<Individual> population){
    std::vector<Individual> newPopulation;
    // на первом шаге, делаем скрещивание 1 элемента с любым, чтобы не потерять лучшего индивида
    while(newPopulation.size()<POPULATION_SIZE&&population.size()>0){
        int parent1ID;
        if(newPopulation.size() == 0) parent1ID = 0;
        else parent1ID = gen()%population.size();
        int parent2ID = gen()%population.size();
        if(parent2ID==parent1ID){
            if(parent2ID>0) parent2ID--;
            else parent2ID++;
        }
        Individual parent1 = population[parent1ID];
        Individual parent2 = population[parent2ID];

        population.erase(population.begin() + std::max(parent1ID, parent2ID));
        population.erase(population.begin() + std::min(parent1ID, parent2ID));
        Individual child1(POLYNOMIAL_DEGREE+1);
        Individual child2(POLYNOMIAL_DEGREE+1);

        int crosspoint = 1 + gen() % (POLYNOMIAL_DEGREE - 1);
        for(int i = 0; i<crosspoint; i++){
            child1[i] = parent1[i];
            child2[i] = parent2[i];
        }
        for(int i = crosspoint; i<=POLYNOMIAL_DEGREE; i++){
            child1[i] = parent2[i];
            child2[i] = parent1[i];
        }
        newPopulation.push_back(parent1);
        newPopulation.push_back(parent2);
        newPopulation.push_back(child1);
        newPopulation.push_back(child2);
    }
    return newPopulation;
}



// Мутация
void Mutation(std::vector<Individual>& population) {
    std::uniform_int_distribution<> dist(0, POLYNOMIAL_DEGREE);
    for (size_t i = 1; i < population.size(); ++i) {
        Individual& individual = population[i];

        if (distr(gen) < MUTATION_RATE) {
            int mutNum = 1 + dist(gen) % 2; // мутируем 1 или 2 гена
            for (int j = 0; j < mutNum; ++j) {
                int bit_to_mutate = dist(gen);  // выбираем случайный ген
                individual[bit_to_mutate] += distr(gen); // добавляем случайное значение
            }
        }
    }
}

double fitn(const Individual& individual){
    double error = 0.0;
    for(int i = 0; i<POINTS.size(); i++){
        double aprox = function_value(individual, POINTS[i]);
        error += std::pow(FUNCTION_VALUES[i]-aprox,2);
    }
    return error/POINTS.size();
}

// Отбор лучших индивидов
std::vector<Individual> Selection(std::vector<Individual> population){
    std::sort(population.begin(),population.end(), [](const Individual& a, const Individual& b){return fitn(a)<fitn(b);});
    return std::vector<Individual>(population.begin(), population.begin() + population.size()*COEF_SELECTION);
}


std::vector<Individual> initPopulation(){
    std::vector<Individual> population;
    for(int i = 0; i < POPULATION_SIZE; i++){
        Individual individual(POLYNOMIAL_DEGREE+1);
        for(double& ind: individual){
            ind = dis(gen);
        }
        population.push_back(individual);
    }
    return population;
}


// Генетический алгоритм
Individual GeneticAlgorithm() {
    // Инициализация начальной популяции
    std::vector<Individual> population = initPopulation();

    double bestFitness = std::numeric_limits<double>::infinity();
    Individual bestIndividual;

    int counterIter = 0;
    for ( int generation = 0; generation < MAX_GENERATIONS;generation++) {
        // Оценка пригодности
        //std::vector<double> fitnesses = Fitness(population);

        // Выбираем лучших особей
        population = Selection(population);

        // Скрещивание и создание нового поколения
        population = Crossover(population);

        // Применяем мутацию
        Mutation(population);

        // Определяем лучшего индивидуума в текущем поколении
        Individual currentIndividual = population[0];
        double currentFitness = fitn(currentIndividual);

        if (currentFitness < bestFitness) {
            counterIter = 0;
            bestFitness = currentFitness;
            bestIndividual = currentIndividual;
        }

        std::cout << "Поколение CPU " << generation << ", Лучшая точность: " << bestFitness << std::endl;
        if(bestFitness < TARGET_ERROR) break;
        counterIter++;
        if(counterIter==maxConstIter) return bestIndividual;
    }

    return bestIndividual;
}

std::vector<Individual> SelectionForGPU(const std::vector<Individual>& population, const std::vector<double>& fitnesses) {
    // Парные значения фитнеса и индивида
    std::vector<std::pair<double, Individual>> fitness_individual_pairs;
    for (int i = 0; i < population.size(); ++i) {
        fitness_individual_pairs.emplace_back(fitnesses[i], population[i]);
    }
    // Сортируем по фитнесу
    std::sort(fitness_individual_pairs.begin(), fitness_individual_pairs.end(), [](const auto& a, const auto& b) { return a.first < b.first; });
    // Выбираем лучших
    int num_selected = static_cast<int>(population.size() * COEF_SELECTION);
    std::vector<Individual> selected_population;
    for (int i = 0; i < num_selected; ++i) {
        selected_population.push_back(fitness_individual_pairs[i].second);
    }
    return selected_population;
}
// Ядро CUDA для вычисления пригодности
__global__ void fitnessKernel(const double* population, double* fitness_scores, const double* points, const double* function_values, int degree, int points_count) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < POPULATION_SIZE) {
        double error = 0.0;
        for (int i = 0; i < points_count; ++i) {
            double approx = 0;
            for(int j = 0; j<=degree; j++){
                approx += population[idx*(degree+1)+j]*pow(points[i],j);
            }
            error += pow(function_values[i] - approx, 2);
        }
        fitness_scores[idx] = error / points_count;
    }
}

void reverse_vector_to_array(double* h_population, std::vector<Individual> population){
    for(int i = 0; i<population.size(); i++){
        for(int j = 0; j<=POLYNOMIAL_DEGREE; j++){
            h_population[i*(POLYNOMIAL_DEGREE+1)+j] = population[i][j];
        }
    }
}

void reverse_array_to_vector(double* h_population, std::vector<Individual>& population){
    for(int i = 0; i<population.size(); i++){
        for(int j = 0; j<=POLYNOMIAL_DEGREE; j++){
            population[i][j] = h_population[i*(POLYNOMIAL_DEGREE+1)+j] ;
        }
    }
}

std::vector<double> evaluateFitnessOnGPU(const std::vector<Individual>& population) {
    double* d_population;
    double* d_fitness_scores;
    double* d_points;
    double* d_function_values;
    double* h_population = new double[(POLYNOMIAL_DEGREE + 1)*POPULATION_SIZE];
    reverse_vector_to_array(h_population,population);
    std::vector<double> fitness_scores(POPULATION_SIZE);

    int individual_size = (POLYNOMIAL_DEGREE + 1) * sizeof(double);
    int population_size = POPULATION_SIZE * individual_size;

    cudaMalloc(&d_population, population_size);
    cudaMalloc(&d_fitness_scores, POPULATION_SIZE * sizeof(double));
    cudaMalloc(&d_points, POINTS.size() * sizeof(double));
    cudaMalloc(&d_function_values, FUNCTION_VALUES.size() * sizeof(double));

    cudaMemcpy(d_population, h_population, population_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_points, POINTS.data(), POINTS.size() * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_function_values, FUNCTION_VALUES.data(), FUNCTION_VALUES.size() * sizeof(double), cudaMemcpyHostToDevice);

    int thread_in_block = 32;
    int blocks = (POPULATION_SIZE + thread_in_block-1) / thread_in_block;
    fitnessKernel<<<blocks, thread_in_block>>>(d_population, d_fitness_scores, d_points, d_function_values, POLYNOMIAL_DEGREE, POINTS.size());
    cudaDeviceSynchronize();

    cudaMemcpy(fitness_scores.data(), d_fitness_scores, POPULATION_SIZE * sizeof(double), cudaMemcpyDeviceToHost);

    cudaFree(d_population);
    cudaFree(d_fitness_scores);
    cudaFree(d_points);
    cudaFree(d_function_values);
    delete[] h_population;

    return fitness_scores;
}

Individual GeneticAlgorithmGPU() {
    // Инициализация начальной популяции
    std::vector<Individual> population = initPopulation();

    double bestFitness = std::numeric_limits<double>::infinity();
    Individual bestIndividual;

    int counterIter = 0;
    for ( int generation = 0; generation < MAX_GENERATIONS; generation++) {
        // Оценка пригодности
        std::vector<double> fitnesses = evaluateFitnessOnGPU(population);

        // Выбираем лучших особей
        population = SelectionForGPU(population, fitnesses);


        // Скрещивание и создание нового поколения
        population = Crossover(population);

        // Применяем мутацию
        Mutation(population);

        Individual currentIndividual  = population[0];
        double currentFitness = fitn(currentIndividual);
        // Определяем лучшего индивидуума в текущем поколении

        if (currentFitness < bestFitness) {
            counterIter = 0;
            bestFitness = currentFitness;
            bestIndividual = currentIndividual;
        }

        std::cout << "Поколение GPU " << generation << ", Лучшая точность: " << bestFitness << std::endl;
        if(bestFitness < TARGET_ERROR) break;
        counterIter++;
        if(counterIter==maxConstIter) return bestIndividual;
    }

    return bestIndividual;
}


int main() {
    initFuncValues();

    auto start_gpu = std::chrono::high_resolution_clock::now();
    Individual best_solution_gpu = GeneticAlgorithmGPU();
    auto end_gpu = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed_gpu = end_gpu - start_gpu;


    /**/
    auto start_cpu = std::chrono::high_resolution_clock::now();
    Individual best_solution = GeneticAlgorithm();
    auto end_cpu = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed_cpu = end_cpu - start_cpu;

    std::cout << "Заданные коэффициенты: ";
    for (double coeff : TARGET_POLYNOM) {
        std::cout << coeff << " ";
    }

    std::cout << std::endl;
    std::cout <<"Точность: "<<fitn(best_solution) << ". Время выполнения CPU: " << elapsed_cpu.count() <<  ". Лучшие коэффициенты полинома CPU: ";
    for (double coeff : best_solution) {
        std::cout << coeff << " ";
    }
    std::cout << std::endl;


    std::cout <<"Точность: "<<fitn(best_solution_gpu)  << ". Время выполнения GPU: " << elapsed_gpu.count() << ". Лучшие коэффициенты полинома GPU: ";
    for (double coeff : best_solution_gpu) {
        std::cout << coeff << " ";
    }
    std::cout << std::endl;

    return 0;
}

Поколение GPU 0, Лучшая точность: 337.843
Поколение GPU 1, Лучшая точность: 180.421
Поколение GPU 2, Лучшая точность: 180.421
Поколение GPU 3, Лучшая точность: 49.5171
Поколение GPU 4, Лучшая точность: 49.5171
Поколение GPU 5, Лучшая точность: 49.5171
Поколение GPU 6, Лучшая точность: 41.7043
Поколение GPU 7, Лучшая точность: 23.1146
Поколение GPU 8, Лучшая точность: 23.1146
Поколение GPU 9, Лучшая точность: 20.2076
Поколение GPU 10, Лучшая точность: 20.2076
Поколение GPU 11, Лучшая точность: 15.5165
Поколение GPU 12, Лучшая точность: 8.45921
Поколение GPU 13, Лучшая точность: 8.45921
Поколение GPU 14, Лучшая точность: 8.45921
Поколение GPU 15, Лучшая точность: 8.45921
Поколение GPU 16, Лучшая точность: 8.45921
Поколение GPU 17, Лучшая точность: 8.45921
Поколение GPU 18, Лучшая точность: 3.42789
Поколение GPU 19, Лучшая точность: 3.26632
Поколение GPU 20, Лучшая точность: 3.26632
Поколение GPU 21, Лучшая точность: 3.26632
Поколение GPU 22, Лучшая точность: 3.19548
Поколение GPU 23, Луч

In [ ]:
%%cuda
// версия в которой происходили эксперементы с лабораторной работой
// не совсем так как описывается в файле методичке
// основное отличие в том, то в новом поколение, я не использую старое
#include <iostream>
#include <vector>
#include <cmath>
#include <algorithm>
#include <random>
#include <limits>

const double left = -5.0; // граница слева
const double right = 5.0; // граница справа
const int number_points = 100; // количество точек
const double step = (right - left)/number_points; // шаг для x
const int POPULATION_SIZE = 1000; // Размер популяции
const int MAX_GENERATION = 1000; // Максимальное количество поколений
const double MUTATION_RATE = 0.01; // Вероятность мутации
const double TARGET_ERROR = 1e-5; // целевая ошибка
const int POLYNOMYAL_DEGREE = 4; // степень полинома
const double COEF_SELECTION = 0.5;


// Точки по которым высчитываем зачение функции
std::vector<double> POINTS;
// Значение функции
std::vector<double> FUNCTION_VALUES;
// Целевой полином, коэффициенты которого хотим найти
std::vector<double> TARGET_POLYNOM(POLYNOMYAL_DEGREE+1);

// Генератор случайных чисел
std::random_device rd;
std::mt19937 gen(rd());
std::uniform_real_distribution<> dis(left, right);
std::uniform_real_distribution<> dis_mutation(0.0, 1.0);
std::uniform_real_distribution<> mutation_dist(0.0, 5.0);


// подсчет значения функции
double function_value(std::vector<double> polynom, float x){
    double result = 0;
    for(int i = 0; i<polynom.size(); i++) {result += polynom[i]*std::pow(x,i);}
    return result;
}

// Инициализация значений (коэффициены целевого полинома, точки, значения фукнций по точкам)
void initFuncValues(){
    for(double& value: TARGET_POLYNOM) value = dis(gen);
    for(double point = left; point < right; point+= step){
        POINTS.push_back(point);
        FUNCTION_VALUES.push_back(function_value(TARGET_POLYNOM,point));
    }
}

// Тип для индивидуума (коэффициенты полинома)
using Individual = std::vector<double>;

// Шаги алгоритма
// Инициализация начальной популяции
// Оценка пригодности (вычисление ошибки)
// Отбор индивидов по значению пригодности
// Скрещивание
// Мутации

std::vector<Individual> initPopulation(){
    std::vector<Individual> population;
    for(int i = 0; i < POPULATION_SIZE; i++){
        Individual individual(POLYNOMYAL_DEGREE+1);
        for(double& ind: individual){
            ind = dis(gen);
        }
        population.push_back(individual);
    }
    return population;
}

// ищем ошибку
double fitness(const Individual& individual){
    double error = 0.0;
    for(int i = 0; i<POINTS.size(); i++){
        double aprox = function_value(individual, POINTS[i]);
        error += std::pow(FUNCTION_VALUES[i]-aprox,2);
    }
    return error/POINTS.size();
}

// отбор
std::vector<Individual> select(std::vector<Individual> population){
    std::sort(population.begin(),population.end(), [](const Individual& a, const Individual& b){return fitness(a)<fitness(b);});
    return std::vector<Individual>(population.begin(), population.begin() + population.size()*COEF_SELECTION);
}

// скрещивание
Individual crossover(const Individual& parent1, const Individual& parent2) {
    Individual child(POLYNOMYAL_DEGREE+1);
    for(int i = 0; i<=POLYNOMYAL_DEGREE; i++){
        if(gen()%2==0){
            child[i] = parent1[i];
        }else{
            child[i] = parent2[i];
        }
    }
    return child;
}


Individual crossover_posl(const Individual& parent1, const Individual& parent2) {
    int cross_point = 1 + gen() % (POLYNOMYAL_DEGREE - 1);
    Individual child(POLYNOMYAL_DEGREE+1);
    for(int i = 0; i<cross_point; i++) child[i] = parent1[i];
    for(int i = cross_point; i<=POLYNOMYAL_DEGREE;i++) child[i] = parent2[i];
    return child;
}


void mutate(Individual& individual, double mean = 0.0, double var = 0.1) {
    std::normal_distribution<> gaussian(mean, var);
    for (double& gene : individual) {
        if (std::uniform_real_distribution<>(0.0, 1.0)(gen) < MUTATION_RATE) {
            gene += gaussian(gen); // Добавляем случайное значение из нормального распределения
        }
    }
}

/*
void mutate_v2(Individual& individual) {
    for (double& coeff : individual) {
        if (dis_mutation(gen) < MUTATION_RATE) {
            coeff *= mutation_dist(gen)*10;
        }
    }
}
*/


Individual geneticAlgorithm(){
    // инициализируем популяцию
    std::vector<Individual> population = initPopulation();
    Individual bestIndividual = population[0];
    double bestFitness = fitness(bestIndividual);

    for(int generation = 0; generation<MAX_GENERATION; generation++){
        // отбираем лучших `родителей`
        population = select(population);

        std::vector<Individual> newPopulation;

        // скрешиваем
        while(newPopulation.size()<POPULATION_SIZE){
            /*int p1Id = gen()%population.size();
            int p2Id = p1Id;
            while(p2Id!=p1Id) p2Id = gen()%population.size();*/

            const Individual& parent1 = population[gen()%population.size()];
            const Individual& parent2 = population[gen()%population.size()];
            // создание ребенка между двумя различными родителями
            Individual child = crossover(parent1, parent2);

            // применяем мутацию
            mutate(child);

            newPopulation.push_back(child);
        }
        population = newPopulation;


        // выбираем лучшего индивида
        Individual currentIndividual = *std::min_element(population.begin(),population.end(), [](const Individual& a, const Individual& b){return fitness(a)<fitness(b);});

        double currentFitness = fitness(currentIndividual);

        if(currentFitness<bestFitness){
            bestFitness = currentFitness;
            bestIndividual = currentIndividual;
        }

        std::cout << "Поколение " << generation << ", Лучшая пригодность: " << bestFitness << std::endl;

        // Остановка, если достигнута целевая ошибка
        if (bestFitness < TARGET_ERROR) break;

    }
    return bestIndividual;
}




int main() {
    //std::cout<<gen<<std::endl;
    //std::cout<<dis(gen)<<std::endl;


    initFuncValues();



    Individual best_solution = geneticAlgorithm();


    std::cout << "Заданные коэффициенты: ";
    for (double coeff : TARGET_POLYNOM) {
        std::cout << coeff << " ";
    }
    std::cout << std::endl;
    std::cout << "Лучшие коэффициенты полинома: ";
    for (double coeff : best_solution) {
        std::cout << coeff << " ";
    }
    std::cout << std::endl;



    return 0;
}

Поколение 0, Лучшая пригодность: 648.345
Поколение 1, Лучшая пригодность: 250.818
Поколение 2, Лучшая пригодность: 57.5431
Поколение 3, Лучшая пригодность: 57.5431
Поколение 4, Лучшая пригодность: 11.9251
Поколение 5, Лучшая пригодность: 11.9251
Поколение 6, Лучшая пригодность: 11.9251
Поколение 7, Лучшая пригодность: 11.9251
Поколение 8, Лучшая пригодность: 10.2995
Поколение 9, Лучшая пригодность: 4.3588
Поколение 10, Лучшая пригодность: 4.3588
Поколение 11, Лучшая пригодность: 4.3588
Поколение 12, Лучшая пригодность: 4.3588
Поколение 13, Лучшая пригодность: 4.3588
Поколение 14, Лучшая пригодность: 4.3588
Поколение 15, Лучшая пригодность: 4.3588
Поколение 16, Лучшая пригодность: 4.3588
Поколение 17, Лучшая пригодность: 4.3588
Поколение 18, Лучшая пригодность: 4.3588
Поколение 19, Лучшая пригодность: 4.3588
Поколение 20, Лучшая пригодность: 4.3588
Поколение 21, Лучшая пригодность: 4.3588
Поколение 22, Лучшая пригодность: 4.3588
Поколение 23, Лучшая пригодность: 4.3588
Поколение 24, Луч